In [ ]:
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor as GBR
from matplotlib import rcParams
import seaborn as sns  # 引入 seaborn 以利用其色彩方案

In [ ]:
# 全局字体大小等设置
global_font_size = 14
font_family = 'serif'
font_serif = ['Times New Roman']
font_weight = 'bold'
# 更新matplotlib全局字体参数

rcParams.update({
    'font.family': font_family,
    'font.serif': font_serif,
    'font.weight': font_weight,
    'font.size': global_font_size,
})


data = pd.read_csv(r'training_set-411.csv')
X = data.iloc[:, 1:-5] 
y = data.iloc[:,-5:]

#标准化特征
stdscale = StandardScaler()
# scaler = MinMaxScaler()
# numerical_features = X.select_dtypes(include='number')
# scaled_numerical_data = scaler.fit_transform(numerical_features)
# X = scaled_numerical_data
# X.replace('none', np.nan, inplace=True)
symbol_columns = ['TM', 'modifications_ring', 'modifications_chain', 'Conjugate_structure','period','group_id']
for column in symbol_columns:
    X[column], _ = pd.factorize(X[column])


for column in X.keys():
    X[column] = (X[column] - X[column].mean()) / X[column].std()

    
# imputer = SimpleImputer(strategy='mean')
# data_filled = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
# X = stdscale.fit_transform(X)stdscale.fit_transform(X[column])

In [ ]:
X

In [ ]:
# 计算相关系数矩阵，包含了任意特征的相关系数，皮尔逊系数
# print('相关系数矩阵为：\n', X.corr())

# 绘制相关性热力图
# plt.subplots(figsize=(8, 8))  # 设置画面大小 
# plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
# plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号 
keys = X.keys()
# 绘制热力图
plt.figure(figsize=(8, 8))
plt.imshow(X.corr(), cmap='coolwarm', interpolation='nearest')
plt.colorbar()
plt.title('Pearson Correlation Coefficient Heatmap', fontweight='bold')
plt.xlabel('Features', fontweight='bold')
plt.ylabel('Features', fontweight='bold')
plt.xticks(np.arange(len(keys)), keys, rotation=90)
plt.yticks(np.arange(len(keys)), keys)
plt.show()

# plt.title('相关性热力图')
# plt.show()


In [ ]:
# 绘制GBR模型中特征的权重

def my_autopct(pct):
    return f'{pct:.1f}%' if pct > 5 else ''#   >5 表示显示占比大于5%的

result = pd.DataFrame(columns=y.columns)

result.insert(0, 'Method', ['SVM', 'Linear Regression', 'Decision Tree Regression', 'Random Forest Regression', 'Gradient Boosting Regression', 'Mean MSE'])
for i in range(5):
    print(y.columns[i], "as label")
    data1 = pd.concat([X, y.iloc[:,i]], axis=1)
    # 删除含有缺失标签的样本
    data_cleaned = data1.dropna(subset=[data1.columns[-1]])

    # 分离特征和标签
    X_cleaned = data_cleaned.iloc[:, :-1]  # 取前n-1列为特征
    y_cleaned = data_cleaned.iloc[:,-1]  # 取最后一列为标签
    keys = X_cleaned.keys()
    X_train, X_test, y_train, y_test = train_test_split(X_cleaned, y_cleaned, test_size=0.2, random_state=42)

    # 初始化梯度提升回归模型
    gb = GBR(learning_rate = 0.1, n_estimators = 50)
    #1{learning_rate = 0.01, n_estimators = 200}
    #3{learning_rate = 0.01, n_estimators = 200}
    #{learning_rate = 0.1, n_estimators = 50}
    
    #gb = RandomForestRegressor(max_depth = 20, min_samples_leaf = 1, min_samples_split = 2, n_estimators = 50)
    #0{max_depth = 20, min_samples_leaf = 1, min_samples_split = 5, n_estimators = 100}
    #2{max_depth = 20, min_samples_leaf = 1, min_samples_split = 2, n_estimators = 50}   
    
    # 拟合模型
    gb.fit(X_train, y_train)

    # 在测试集上做预测
    y_pred = gb.predict(X_test)
    y_out = gb.predict(X_train)

    # 评估模型性能
    mse = mean_squared_error(y_test, y_pred)
    trainR2 = r2_score(y_train, y_out)
    testR2 = r2_score(y_test, y_pred)
    fig, ax1 = plt.subplots()
    print("Gradient Boosting Regression \n Mean Squared Error: ", mse, "train R2: ", trainR2, "test R2: ", testR2)
    x = np.append(y_train, y_test)
    ax1.scatter(y_train, y_out, color='turquoise', label='Train data',alpha=0.7)
    ax1.scatter(y_test, y_pred, color='deeppink', label='Test data',alpha=0.7)
    text_props = {'ha': 'right', 'va': 'bottom', 'fontsize': 12}
    ax1.text(0.99, 0.11, f"Training R$^{2}$: {trainR2:.2f}", transform=ax1.transAxes, **text_props)
    ax1.text(0.99, 0.06, f"Test R$^{2}$: {testR2:.2f}", transform=ax1.transAxes, **text_props)
    ax1.text(0.99, 0.01, f"MSE: {mse:.2f}", transform=ax1.transAxes, **text_props)
    ax1.legend(loc='upper left',  frameon=False)
    #ax1.plot(x, x, color='gray', linewidth=2, linestyle='--')
    ax1.plot(ax1.get_xlim(), ax1.get_xlim(), color='gray', linewidth=2, linestyle='--')
    plt.xlabel("E(DFT) (eV)", fontweight='bold')
    plt.ylabel('E(ML) (eV)', fontweight='bold')
    plt.show()

    feature_importance = gb.feature_importances_
    custom_colors =[(0.12156862745098039, 0.4666666666666667, 0.7058823529411765), (0.6823529411764706, 0.7803921568627451, 0.9098039215686274),
     (1.0, 0.4980392156862745, 0.054901960784313725), (1.0, 0.7333333333333333, 0.47058823529411764),
      (0.17254901960784313, 0.6274509803921569, 0.17254901960784313), 
      (0.596078431372549, 0.8745098039215686, 0.5411764705882353), (0.8392156862745098, 0.15294117647058825, 0.1568627450980392), (1.0, 0.596078431372549, 0.5882352941176471),
       (0.5803921568627451, 0.403921568627451, 0.7411764705882353), (0.7725490196078432, 0.6901960784313725, 0.8352941176470589), (0.5490196078431373, 0.33725490196078434, 0.29411764705882354),
        (0.7686274509803922, 0.611764705882353, 0.5803921568627451), (0.8901960784313725, 0.4666666666666667, 0.7607843137254902), (0.9686274509803922, 0.7137254901960784, 0.8235294117647058),
         (0.4980392156862745, 0.4980392156862745, 0.4980392156862745), (0.7803921568627451, 0.7803921568627451, 0.7803921568627451), (0.7372549019607844, 0.7411764705882353, 0.13333333333333333),
          (0.8588235294117647, 0.8588235294117647, 0.5529411764705883), (0.09019607843137255, 0.7450980392156863, 0.8117647058823529), (0.6196078431372549, 0.8549019607843137, 0.8980392156862745)
          ,'orangered', 'red', 'mediumslateblue']
    #custom_colors = ['darkorchid', 'yellow', 'orangered', 'red', 'cyan', 'lawngreen', 'lightgray', 'pink', 'slateblue',  
    #                'gold', 'lightcoral', 'azure', 'tomato', 'palegreen', 'brown', 'mediumslateblue', 'cornflowerblue', 
    #                'dodgerblue', 'darksalmon', 'deepskyblue', 'violet', 'darkblue', 'peachpuff', 'powderblue', 'maroon', 'hotpink']
    #custom_colors = sns.color_palette('tab20')  # Replace 'muted' with your chosen palette
    #print(custom_colors)
    # 绘制饼状图
    plt.figure(figsize=(23, 23))
    patches, texts, autotexts = plt.pie(feature_importance, autopct=my_autopct, startangle=140, textprops={'fontsize': 50}, pctdistance=0.7,colors=custom_colors)
    # 画饼状图textprops={'fontsize': 35}字体大小；pctdistance=0.7饼状图中的百分数离圆心的距离
    plt.legend(patches, [f'{name} ({value*100:.1f}%)' for name, value in zip(keys, feature_importance)],frameon=False, loc="center left", bbox_to_anchor=(1, 0.5), prop={'size':35})
    # 标签prop={'size':35}字体大小；
    plt.axis('equal')
    plt.title("Feature Importance Pie Chart of "+ y.columns[i], fontsize=50)
    fn = "Feature Importance Pie Chart of "+ y.columns[i] + ".png"
#     plt.savefig(fn, dpi=300) 
    plt.show()

In [ ]:
def TrainSVM(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    svr = SVR(kernel='rbf', C=100, gamma=0.01 )    # linear sigmoid rbf poly
    svr.fit(X_train, y_train)
    y_pred = svr.predict(X_test)
    y_out = svr.predict(X_train)
    mse = mean_squared_error(y_test, y_pred)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    trainR2 = r2_score(y_train, y_out)
    testR2 = r2_score(y_test, y_pred)
    fig, ax1 = plt.subplots()
    print("SVM \n Mean Squared Error: ", mse, "train R2: ", trainR2, "test R2: ", testR2)
    x = np.append(y_train, y_test)
    ax1.scatter(y_train, y_out, color='turquoise', label='Train data',alpha=0.7)
    ax1.scatter(y_test, y_pred, color='deeppink', label='Test data',alpha=0.7)
    text_props = {'ha': 'right', 'va': 'bottom', 'fontsize': 12}
    ax1.text(0.99, 0.11, f"Training R$^{2}$: {trainR2:.2f}", transform=ax1.transAxes, **text_props)
    ax1.text(0.99, 0.06, f"Test R$^{2}$: {testR2:.2f}", transform=ax1.transAxes, **text_props)
    ax1.text(0.99, 0.01, f"RMSE: {test_rmse:.2f}", transform=ax1.transAxes, **text_props)
    ax1.legend(loc='upper left',  frameon=False)
    #ax1.plot(x, x, color='gray', linewidth=2, linestyle='--')
    ax1.plot(ax1.get_xlim(), ax1.get_xlim(), color='gray', linewidth=2, linestyle='--')
    plt.xlabel("E(DFT) (eV)", fontweight='bold')
    plt.ylabel('E(ML) (eV)', fontweight='bold')
    plt.show()
    return mse

In [ ]:
def TrainLinReg(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=28)

    # 初始化线性回归模型
    lr = LinearRegression()

    # 拟合模型
    lr.fit(X_train, y_train)

    # 在测试集上做预测
    y_pred = lr.predict(X_test)
    y_out = lr.predict(X_train)

    # 评估模型性能
    mse = mean_squared_error(y_test, y_pred)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    trainR2 = r2_score(y_train, y_out)
    testR2 = r2_score(y_test, y_pred)
    fig, ax1 = plt.subplots()
    print("Linear Regression \n Mean Squared Error: ", mse, "train R2: ", trainR2, "test R2: ", testR2)
    x = np.append(y_train, y_test)
    ax1.scatter(y_train, y_out, color='turquoise', label='Train data',alpha=0.7)
    ax1.scatter(y_test, y_pred, color='deeppink', label='Test data',alpha=0.7)
    text_props = {'ha': 'right', 'va': 'bottom', 'fontsize': 12}
    ax1.text(0.99, 0.11, f"Training R$^{2}$: {trainR2:.2f}", transform=ax1.transAxes, **text_props)
    ax1.text(0.99, 0.06, f"Test R$^{2}$: {testR2:.2f}", transform=ax1.transAxes, **text_props)
    ax1.text(0.99, 0.01, f"RMSE: {test_rmse:.2f}", transform=ax1.transAxes, **text_props)
    ax1.legend(loc='upper left',  frameon=False)
    #ax1.plot(x, x, color='gray', linewidth=2, linestyle='--')
    ax1.plot(ax1.get_xlim(), ax1.get_xlim(), color='gray', linewidth=2, linestyle='--')
    plt.xlabel("E(DFT) (eV)", fontweight='bold')
    plt.ylabel('E(ML) (eV)', fontweight='bold')
    plt.show()
    return mse

In [ ]:
def TrainDeciTree(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 初始化决策树回归模型
    dt = DecisionTreeRegressor()

    # 拟合模型
    dt.fit(X_train, y_train)

    # 在测试集上做预测
    y_pred = dt.predict(X_test)
    y_out = dt.predict(X_train)

    # 评估模型性能
    mse = mean_squared_error(y_test, y_pred)
    trainR2 = r2_score(y_train, y_out)
    testR2 = r2_score(y_test, y_pred)
    fig, ax1 = plt.subplots()
    print("Decision Tree Regression \n Mean Squared Error: ", mse, "train R2: ", trainR2, "test R2: ", testR2)
    x = np.append(y_train, y_test)
    ax1.scatter(y_train, y_out, color='turquoise', label='Train data',alpha=0.7)
    ax1.scatter(y_test, y_pred, color='deeppink', label='Test data',alpha=0.7)
    text_props = {'ha': 'right', 'va': 'bottom', 'fontsize': 12}
    ax1.text(0.99, 0.11, f"Training R$^{2}$: {trainR2:.2f}", transform=ax1.transAxes, **text_props)
    ax1.text(0.99, 0.06, f"Test R$^{2}$: {testR2:.2f}", transform=ax1.transAxes, **text_props)
    ax1.text(0.99, 0.01, f"MSE: {mse:.2f}", transform=ax1.transAxes, **text_props)
    ax1.legend(loc='upper left',  frameon=False)
    #ax1.plot(x, x, color='gray', linewidth=2, linestyle='--')
    ax1.plot(ax1.get_xlim(), ax1.get_xlim(), color='gray', linewidth=2, linestyle='--')
    plt.xlabel("E(DFT) (eV)", fontweight='bold')
    plt.ylabel('E(ML) (eV)', fontweight='bold')
    plt.show()
    return mse

In [ ]:
def TrainRandForest(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 初始化随机森林回归模型
    rf = RandomForestRegressor()

    # 拟合模型
    rf.fit(X_train, y_train)

    # 在测试集上做预测
    y_pred = rf.predict(X_test)
    y_out = rf.predict(X_train)

    # 评估模型性能
    mse = mean_squared_error(y_test, y_pred)
    trainR2 = r2_score(y_train, y_out)
    testR2 = r2_score(y_test, y_pred)
    fig, ax1 = plt.subplots()
    print("Random Forest Regression \n Mean Squared Error: ", mse, "train R2: ", trainR2, "test R2: ", testR2)
    x = np.append(y_train, y_test)
    ax1.scatter(y_train, y_out, color='turquoise', label='Train data',alpha=0.7)
    ax1.scatter(y_test, y_pred, color='deeppink', label='Test data',alpha=0.7)
    text_props = {'ha': 'right', 'va': 'bottom', 'fontsize': 12}
    ax1.text(0.99, 0.11, f"Training R$^{2}$: {trainR2:.2f}", transform=ax1.transAxes, **text_props)
    ax1.text(0.99, 0.06, f"Test R$^{2}$: {testR2:.2f}", transform=ax1.transAxes, **text_props)
    ax1.text(0.99, 0.01, f"MSE: {mse:.2f}", transform=ax1.transAxes, **text_props)
    ax1.legend(loc='upper left',  frameon=False)
    #ax1.plot(x, x, color='gray', linewidth=2, linestyle='--')
    ax1.plot(ax1.get_xlim(), ax1.get_xlim(), color='gray', linewidth=2, linestyle='--')
    plt.xlabel("E(DFT) (eV)", fontweight='bold')
    plt.ylabel('E(ML) (eV)', fontweight='bold')
    plt.show()
    return mse

In [ ]:
def TrainBoost(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

    # 初始化梯度提升回归模型
    # n_estimators=100, max_depth=3
    gb = GBR(n_estimators=15, max_depth=3, learning_rate=0.2)

    # 拟合模型
    gb.fit(X_train, y_train)

    # 在测试集上做预测
    y_pred = gb.predict(X_test)
    y_out = gb.predict(X_train)

    # 评估模型性能
    mse = mean_squared_error(y_test, y_pred)
    trainR2 = r2_score(y_train, y_out)
    testR2 = r2_score(y_test, y_pred)
    fig, ax1 = plt.subplots()
    print("Gradient Boosting Regression \n Mean Squared Error: ", mse, "train R2: ", trainR2, "test R2: ", testR2)
    x = np.append(y_train, y_test)
    ax1.scatter(y_train, y_out, color='turquoise', label='Train data',alpha=0.7)
    ax1.scatter(y_test, y_pred, color='deeppink', label='Test data',alpha=0.7)
    text_props = {'ha': 'right', 'va': 'bottom', 'fontsize': 12}
    ax1.text(0.99, 0.11, f"Training R$^{2}$: {trainR2:.2f}", transform=ax1.transAxes, **text_props)
    ax1.text(0.99, 0.06, f"Test R$^{2}$: {testR2:.2f}", transform=ax1.transAxes, **text_props)
    ax1.text(0.99, 0.01, f"MSE: {mse:.2f}", transform=ax1.transAxes, **text_props)
    ax1.legend(loc='upper left',  frameon=False)
    #ax1.plot(x, x, color='gray', linewidth=2, linestyle='--')
    ax1.plot(ax1.get_xlim(), ax1.get_xlim(), color='gray', linewidth=2, linestyle='--')
    plt.xlabel("E(DFT) (eV)", fontweight='bold')
    plt.ylabel('E(ML) (eV)', fontweight='bold')
    plt.show()
    return mse

In [ ]:
result = pd.DataFrame(columns=y.columns)
result.insert(0, 'Method', ['SVM', 'Linear Regression', 'Decision Tree Regression', 'Random Forest Regression', 'Gradient Boosting Regression', 'Mean MSE'])
for i in range(5):
    print(y.columns[i], "as label")
    data1 = pd.concat([X, y.iloc[:,i]], axis=1)
    # 删除含有缺失标签的样本
    data_cleaned = data1.dropna(subset=[data1.columns[-1]])

    # 分离特征和标签
    X_cleaned = data_cleaned.iloc[:, :-1]  # 取前n-1列为特征
    y_cleaned = data_cleaned.iloc[:,-1]  # 取最后一列为标签

    mseSVM = TrainSVM(X_cleaned, y_cleaned)
    result.iloc[0, i+1] = mseSVM
    mseLinReg = TrainLinReg(X_cleaned, y_cleaned)
    result.iloc[1, i+1] = mseLinReg
    mseDeciTree = TrainDeciTree(X_cleaned, y_cleaned)
    result.iloc[2, i+1] = mseDeciTree
    mseRandForest = TrainRandForest(X_cleaned, y_cleaned)
    result.iloc[3, i+1] = mseRandForest
    mseBoost = TrainBoost(X_cleaned, y_cleaned)
    result.iloc[4, i+1] = mseBoost
#     mseBP = TrainBP(X_cleaned, y_cleaned)
    mseAvg = (mseSVM + mseLinReg + mseDeciTree + mseRandForest + mseBoost) / 5
    result.iloc[5, i+1] = mseAvg
    print("Average MSE: ", mseAvg)
    print("-------------------------------------------------------")
    
result.to_csv('res.csv', index=False)
